# Treinamento do modelo preditivo 

## setup

In [15]:
# conexão com o Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
#importação de bibliotecas
import pandas as pd 
import numpy as np
import pickle
!pip install pycaret
from pycaret.classification import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
#criação de data frame em pandas
df = pd.read_csv('/content/drive/MyDrive/hackathon/new_dataframe_c.csv')
df

,vlr_credito,vlr_score,rating,nota
0,0.003473,0.377,1.0,3
1,0.000120,0.773,1.0,3
2,0.007876,0.560,1.0,3
3,0.010912,0.374,1.0,3
4,0.000948,0.611,1.0,3
...,...,...,...,...
5108524,0.000113,0.575,1.0,3
5108525,0.003511,0.538,1.0,3
5108526,0.004161,0.437,1.0,3
5108527,0.002122,0.410,0.0,1


Separação de dados em treino e teste

In [18]:
#seleção aleatoria de 70000 dados para teste e 30000  para treino
train_idx = np.random.choice(df.index, size=70000, replace=False)
test_idx = np.random.choice(df.drop(train_idx).index, size=30000, replace=False)

# criar os conjuntos de treino e teste
train_df = df.iloc[train_idx]
test_df = df.iloc[test_idx]

In [19]:
clf = setup(data=train_df, target='nota')

# Criando um modelo 
lr = create_model('lr')

# Analize do modelo 
tuned_lr = tune_model(lr)

# Realizando a previsão com base no treino e teste
predictions = predict_model(tuned_lr, data=test_df)

# Salvando o modelo 
save_model(tuned_lr, 'modelo_lr')

,Description,Value
0,Session id,8600
1,Target,nota
2,Target type,Multiclass
3,Target mapping,"1: 0, 2: 1, 3: 2, 4: 3"
4,Original data shape,"(70000, 4)"
5,Transformed data shape,"(70000, 4)"
6,Transformed train set shape,"(49000, 4)"
7,Transformed test set shape,"(21000, 4)"
8,Numeric features,3
9,Preprocess,True


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9035,0.9769,0.9035,0.9017,0.9021,0.7807,0.7815
1,0.9055,0.9777,0.9055,0.9036,0.9041,0.7852,0.7861
2,0.9090,0.9784,0.9090,0.9068,0.9072,0.7920,0.7933
3,0.9078,0.9751,0.9078,0.9056,0.9058,0.7886,0.7901
4,0.9004,0.9769,0.9004,0.8978,0.8980,0.7711,0.7728
5,0.9082,0.9776,0.9082,0.9068,0.9071,0.7923,0.7929
6,0.9057,0.9766,0.9057,0.9041,0.9044,0.7861,0.7869
7,0.9018,0.9747,0.9018,0.8993,0.8999,0.7767,0.7777
8,0.9088,0.9766,0.9088,0.9072,0.9076,0.7934,0.7941


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9037,0.9772,0.9037,0.9017,0.9024,0.7827,0.7832
1,0.9073,0.9779,0.9073,0.9057,0.9062,0.7908,0.7914
2,0.9102,0.9786,0.9102,0.9082,0.9087,0.7963,0.7971
3,0.9100,0.9755,0.9100,0.9078,0.9082,0.7951,0.7962
4,0.9010,0.9772,0.9010,0.8987,0.8992,0.7740,0.7752
5,0.9076,0.9778,0.9076,0.9063,0.9067,0.7923,0.7926
6,0.9073,0.9770,0.9073,0.9058,0.9063,0.7911,0.7915
7,0.9022,0.9750,0.9022,0.8998,0.9005,0.7787,0.7794
8,0.9065,0.9767,0.9065,0.9049,0.9055,0.7898,0.7902


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.9067,0.9777,0.9044,0.9007,0.9020,0.7879,0.7888


Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=FastMemory(location=/tmp/joblib),
          steps=[('label_encoding',
                  TransformerWrapperWithInverse(exclude=None, include=None,
                                                transformer=LabelEncoder())),
                 ('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['vlr_credito', 'vlr_score',
                                              'rating'],
                                     transformer=SimpleImputer(add_indicator=False,
                                                               copy=True,
                                                               fill_value=None,
                                                               keep_empty_features=F...
                                                               missing_values=nan,
                                                               strategy='most_frequent',
                                        

In [20]:
#função de teste para testar o modelo
def chamar_modelo(dados):
  # Carregando o modelo
  saved_lr = load_model('modelo_lr')

  # tranformando os dados no formato correto 
  novos_dados = dados
  novos_dados_df = pd.DataFrame(novos_dados)

  #realizando nova previsão
  new_predictions = predict_model(saved_lr, data=novos_dados_df)
  resultado = (new_predictions['prediction_label'][0])
  print(resultado)

In [21]:
#chamada de exemplo
chamar_modelo([{'vlr_credito': 0.001, 'vlr_score': 0.9, 'rating': 0.5}])

Transformation Pipeline and Model Successfully Loaded


3


In [22]:
# Salva o modelo treinado em um arquivo .pkl 
file_path = '/content/drive/My Drive/Colab Notebooks/modelo_lr.pkl'

with open(file_path, 'wb') as file:
    pickle.dump(tuned_lr, file)